Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [X] Continue to participate in our Kaggle challenge. 
- [X] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [X] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [X] Add comments and Markdown to your notebook. Clean up your code.
- [X] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [13]:
### Select features, scikit-learn pipeline to encode categoricals, impute missing values, and fit a decision tree classifier.
import category_encoders as ce

import xgboost as xgb
from xgboost import XGBClassifier

from scipy.stats import randint, uniform

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

import eli5
from eli5.sklearn import PermutationImportance

### Wrangle function
def wrangle(df):
    """I heard you like docstrings"""
    import numpy as np
    
    ### Easy stuff
    # Copy the dataframe to itself so warnings can shsshshsh
    df = df.copy()
    
    df['latitude'] = df['latitude'].replace(-2e-08,0)
    df['date_recorded'] = pd.to_datetime(df['date_recorded'],infer_datetime_format=True)
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df['day_recorded'] = df['date_recorded'].dt.day
    
    # Drop dupes
    df = df.drop(columns=['id','date_recorded','quantity_group',
                          'scheme_management','quality_group',
                          'waterpoint_type_group','extraction_type_group'])

    # Ryan's Engineer feature: how many years from construction_year to date_recorded
    df['years'] = df['year_recorded'] - df['construction_year']
    df['years_MISSING'] = df['years'].isnull()

    # YEEHAW dataframe return
    return df

### Do this cool permutation importance thing
def permutationCreator(X_train, y_train):
    ## Transform manually cause pipelines don't work
    transformers = make_pipeline(
        ce.OrdinalEncoder(),
        SimpleImputer(strategy='mean')
    )
    X_train_t = transformers.fit_transform(X_train)
        
    ## Create model and fit
    # n_esties 550
    model = RandomForestClassifier(n_estimators=500,random_state=4,n_jobs=-1)
    model.fit(X_train_t, y_train)
    
    ## Fit permutation stuff
    permuter = PermutationImportance(
        model,
        scoring = 'accuracy',
        n_iter = 4,
        random_state = 4
    )
    permuter.fit(X_train_t,y_train)
    features = X_train.columns.tolist()
    return permuter, features

### Prune by feature importance
def featureImportance(train, permuter, min_imp):
    mask = permuter.feature_importances_ > min_imp
    features = train.columns[mask]
    X_train = train[features]
    
    return X_train, features

### Get that cool green/red graph
def permutationColor(permuter, features):
    x = eli5.show_weights(
        permuter, feature_names=features, top=None
    )
    return x

###########################################################
##### !!!!
### Random Forest w/ Ordinal Encoder
rf = make_pipeline(
    ce.OrdinalEncoder(),
    #ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    # n_estimators is # of trees in forest
    ## n_estimators=650
    RandomForestClassifier(n_estimators=1000,random_state=4,n_jobs=-1)
)

### XGB Classifier!!!
### (does not have a score method: use accuracy_score)
xgb = make_pipeline(
    ce.OrdinalEncoder(),
    # No special Early Stopping
    XGBClassifier(n_estimators=100,n_jobs=-1)
)

In [14]:
### Cross-validation Pipeline
pipeline = make_pipeline(
    # All params are going to be in a separate dictionary below X_train
    ce.TargetEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=4)
)

### Make target
target = 'status_group'
# Assign variables
X_train = wrangle(train)
X_train = X_train.drop(target,axis=1)
y_train = train[target]

#features = X_train.columns.tolist()
#X_test = wrangle(test)
status_map = {'functional needs repair':1,'functional':2,'non functional':0}
y_train = y_train.map(status_map)

## Get permuter
permuter, features_old = permutationCreator(X_train, y_train)

### Get feature importances
X_train, features = featureImportance(X_train, permuter, .0005)

### Pretty permutation colours!
permutationColor(permuter, features_old)

Weight,Feature
0.1578 ± 0.0011,quantity
0.0370 ± 0.0010,longitude
0.0314 ± 0.0010,latitude
0.0283 ± 0.0011,amount_tsh
0.0239 ± 0.0005,waterpoint_type
0.0215 ± 0.0004,wpt_name
0.0180 ± 0.0005,subvillage
0.0167 ± 0.0005,population
0.0144 ± 0.0006,years
0.0142 ± 0.0005,extraction_type


In [15]:
print(X_train.columns)

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'wpt_name', 'basin', 'subvillage', 'region',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_name', 'construction_year', 'extraction_type',
       'extraction_type_class', 'management', 'payment', 'payment_type',
       'water_quality', 'quantity', 'source', 'source_type', 'waterpoint_type',
       'month_recorded', 'day_recorded', 'years'],
      dtype='object')


In [16]:
param_dist = {
    'targetencoder__min_samples_leaf': randint(1,500),
    'targetencoder__smoothing': uniform(1,600),
    'simpleimputer__strategy': ['mean','median'],
    'randomforestclassifier__n_estimators': randint(100,600,10),
    'randomforestclassifier__max_depth': [25, 50, 100, 150, None], 
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    # Randomly try 100 options sampling
    n_iter=60,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

print(train.status_group.unique())
print(X_train.shape,y_train.shape)
print(X_train.columns)
search.fit(X_train,y_train)

['functional' 'non functional' 'functional needs repair']
(59400, 30) (59400,)
Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'wpt_name', 'basin', 'subvillage', 'region',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_name', 'construction_year', 'extraction_type',
       'extraction_type_class', 'management', 'payment', 'payment_type',
       'water_quality', 'quantity', 'source', 'source_type', 'waterpoint_type',
       'month_recorded', 'day_recorded', 'years'],
      dtype='object')
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   47.0s
/home/ygeun/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed: 10.4min
[Parallel(n_jobs

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('targetencoder',
                                              TargetEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            min_samples_leaf=1,
                                                            return_df=True,
                                                            smoothing=1.0,
                                                            verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                           

In [17]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__max_depth,param_randomforestclassifier__n_estimators,param_simpleimputer__strategy,param_targetencoder__min_samples_leaf,param_targetencoder__smoothing,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,45.389355,0.373729,1.700309,0.042992,25,332,mean,384,35.4303,"{'randomforestclassifier__max_depth': 25, 'ran...",...,0.794764,0.001666,1,0.998632,0.998822,0.998506,0.998359,0.998737,0.998611,0.000165
7,59.281789,0.297904,2.191856,0.156729,25,440,mean,382,42.4894,"{'randomforestclassifier__max_depth': 25, 'ran...",...,0.793973,0.001729,2,0.998401,0.998527,0.998359,0.998148,0.998590,0.998405,0.000153
27,56.596784,0.825183,1.946275,0.048123,25,402,mean,163,66.0322,"{'randomforestclassifier__max_depth': 25, 'ran...",...,0.793350,0.002232,3,0.996254,0.996885,0.996970,0.996928,0.996907,0.996789,0.000269
0,26.778849,0.168925,1.011539,0.010194,150,184,mean,324,39.0136,"{'randomforestclassifier__max_depth': 150, 'ra...",...,0.792912,0.002050,4,0.999874,0.999937,0.999916,0.999937,1.000000,0.999933,0.000041
57,23.559928,0.555677,0.946054,0.058719,150,139,mean,28,11.8892,"{'randomforestclassifier__max_depth': 150, 'ra...",...,0.792912,0.002195,4,0.999853,0.999937,0.999916,0.999937,1.000000,0.999928,0.000047


In [18]:
pipeline = search.best_estimator_
X_test = wrangle(test)
X_test = X_test[features]
#y_test = X_test[target]
#status_map = {2:'functional',1:'functional needs repair',0:'non functional'}
#y_test['status_group'] = y_test['status_group'].map(status_map)
y_pred = pipeline.predict(X_test)

In [19]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.head()
#''';

,id,status_group
0,50785,2
1,51630,2
2,17168,2
3,45559,0
4,49871,2


In [20]:
status_map = {1:'functional needs repair',2:'functional',0:'non functional'}
submission.status_group = submission.status_group.map(status_map)
submission.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [21]:
submission.to_csv('../../sub-01.csv', index=False)